In [5]:
import dask
import pystac_client
import planetary_computer
import leafmap
import geogif
import stackstac
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from avgNdwi import main

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 13.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37965,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 13.65 GiB
Comm: tcp://127.0.0.1:39845,Total threads: 2
Dashboard: http://127.0.0.1:41013/status,Memory: 3.41 GiB
Nanny: tcp://127.0.0.1:44969,


In [3]:
bbox = (32.16033157094111, 22.852984440390316, 33.288140399555346, 23.806193394664234)
start = ['2017']
end = ['2018']

In [6]:
%%time
main(bbox, start, end)

CPU times: user 4 μs, sys: 1 μs, total: 5 μs
Wall time: 8.58 μs
